In [ ]:
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.python.client import device_lib 
from tensorflow.keras.datasets import cifar10

tf.__version__

In [ ]:
# See where tensorflow Cpu / Gpu is running
print(device_lib.list_local_devices())

In [ ]:
# define class names

class_names = ['Plane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Boat', 'Truck']

In [ ]:
# Load and split Dataset 

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:
# Normalize Dataset

X_train = X_train / 255.0

X_test = X_test / 255.0

In [ ]:
# We have 50,000 images of 32x32 pixels with 3 colors

X_train.shape

In [ ]:
# example image

plt.imshow(X_test[20])

In [ ]:
class_names[int(y_test[20])]

In [ ]:
# We define the Convolutional neural network model
model = tf.keras.models.Sequential()

# Add the first convolutional layer
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=[32, 32, 3]))

# Add the second convolutional layer
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"))

# Add max pooling layer
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

# Add the third convolutional layer
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))

# Add the quarter convolutional layer
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))

# Add another max pooling layer
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

# Add flattening layer
model.add(tf.keras.layers.Flatten())

# Add the output layer
# units: number of classes (10 in the case of cifar), activation function: 'softmax' (probabilities of each class)
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

# Copile the model
# Use adam optimizer and loss with Sparse softmax (categorical) crossentropy 
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

model.summary()

In [ ]:
# We train the model

model.fit(X_train, y_train, epochs=5)

In [ ]:
# Test the model

test_loss, test_accuracy = model.evaluate(X_test, y_test)

In [ ]:
# Save the model

model_json = model.to_json()
with open("cifar10.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
# Save model weight

model.save_weights("fashion_model.h5")

In [ ]:
# We re-train the model 5 epoch more

model.fit(X_train, y_train, epochs=5)

In [ ]:
# re-Test the model with the new training

test_loss, test_accuracy = model.evaluate(X_test, y_test)

In [ ]:
# the model work worst because now are overfitting